## 코사인 유사도를 이용해서 문서의 유사도를 구해보자.

### 1. 코사인 유사도(Cosine Similarity)

두 벡터간 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도를 의미한다.  
두 벡터의 방향이 완전히 같다면 1,  
두 벡터의 방향이 90도면 0,  
두 벡터의 방향이 180도로 반대라면 -1

$similarity=cos(Θ)=\frac{A⋅B}{||A||\ ||B||}=\frac{\sum_{i=1}^{n}{A_{i}×B_{i}}}{\sqrt{\sum_{i=1}^{n}(A_{i})^2}×\sqrt{\sum_{i=1}^{n}(B_{i})^2}}$

In [3]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print('문서 1과 문서 2의 유사도 : ', cos_sim(doc1,doc2))
print('문서 1과 문서 3의 유사도 : ', cos_sim(doc1,doc3))
print('문서 2과 문서 3의 유사도 : ', cos_sim(doc2,doc3))

문서 1과 문서 2의 유사도 :  0.6666666666666667
문서 1과 문서 3의 유사도 :  0.6666666666666667
문서 2과 문서 3의 유사도 :  1.0000000000000002


문서1 문서2 유사도와, 문서1 문서3의 유사도가 같다.  
문서2 문서3의 유사도가 1이 나온다.  

코사인 유사도는 유사도를 구할 때 벡터의 방향(패턴)에 초점을 두므로 코사인 유사도는 문서의 길이가 다른 상황에서 비교적 공정한 비교를 할 수 있도록 도와준다.  

### 2. 유사도를 이용한 추천 시스템 구현하기

TF-IDF와 코사인 유사도만으로 영화의 줄거리에 기반해서 영화를 추천하는 추천 시스템을 만들어보자.

#### kaggle에서 dataset donwload

In [5]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("rounakbanik/the-movies-dataset")

print("Path to dataset files:", path)

c:\Users\user\Workspace\deeplearning-pytorch-textbook\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 228M/228M [00:08<00:00, 27.2MB/s] 

Extracting files...


Path to dataset files: C:\Users\user\.cache\kagglehub\datasets\rounakbanik\the-movies-dataset\versions\7


#### 경로 설정

In [6]:
import os
data_path = r'C:\Users\user\.cache\kagglehub\datasets\rounakbanik\the-movies-dataset\versions\7'
file_name = 'movies_metadata.csv'
full_path = os.path.join(data_path,file_name)

#### csv 파일로 데이터셋 파일 읽기

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv(full_path, low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


영화 제목에 해당하는 title열과 줄거리에 해당하는 overview열을  
코사인 유사도에 활용해보자.  

좋아하는 영화를 입력하면,  
해당 영화의 줄거리와 유사한 줄거리의 영화를 찾아서 추천하는 시스템을 만들자.  

훈련 데이터의 양을 줄이고 학습을 진행하기 위해 아래와 같이 재저장한다.

In [8]:
# 상위 2만개의 샘플을 data에 저장
data = data.head(20000)

결측값을 확인해보자.

In [ ]:
# overview 열에 존재하는 모든 결측값을 전부 카운트하여 출력
print('overview 열의 결측값의 수 : ', data['overview'].isnull().sum())

overview 열의 결측값의 수 :  135


결측값을 빈 값으로 대체한다.

In [10]:
# 결측값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('')

overview열에 대해서 TF-IDF 행렬을 구한 후 행렬의 크기를 출력해보자.  

TF-IDF 행렬의 크기(shape)는 (20000, 47487)이다.  
이는 즉 20,000개의 영화를 표현하기 위해서, 총 47,487개의 단어를 사용했다는 뜻이다.  
혹은 47,487차원의 문서 벡터가 20,000개 존재한다고 표현할 수도 있다.

In [ ]:
# TfidfVectorizer : Convert a collection of raw documents to a matrix of TF-IDF features.
tfidf = TfidfVectorizer(stop_words='english')
# fit_transform : Learn vocabulary and idf, return document-term matrix.
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) : ', tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) :  (20000, 47487)


이제 코사인 유사도 연산을 수행해보자.  

`cosine_sim`은 자기 자신에 대한 상호 벡터 간의 유사도를 기록한 행렬이다. 

In [12]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :', cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


이제 기존 데이터프레임으로부터 영화의 타이틀을 key, 영화의 인덱스를 value로 하는 딕셔너리 title_to_index를 만들어보자.

In [13]:
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
idx

4

선택한 영화의 제목을 입력하면 코사인 유사도를 통해 가장 overview가 유사한 10개의 영화를 찾아내는 함수를 만든다.  

In [14]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    # iloc 행번호로 선택
    return data['title'].iloc[movie_indices]

In [15]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

In [16]:
get_recommendations('Avatar')

3657             Project Moon Base
18444                    Apollo 18
18002        The War of the Robots
15706                 The American
16068                Bloodbrothers
19943    Welcome to the Space Show
2458                    The Matrix
7043                       The Men
13887         The Inhabited Island
603          Hellraiser: Bloodline
Name: title, dtype: object